# imports

In [2]:
! pip install stanza edist

In [33]:
import networkx as nx
from networkx.drawing import nx_agraph
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import numpy as np
import re

import stanza

import nltk
from nltk.tree import Tree

import edist.tree_utils as tree_utils
import edist.tree_edits as tree_edits
import edist.ted as ted

In [5]:
# Build a Neural Pipeline
stanza.download('en')
StanzaPipeline = stanza.Pipeline('en', processors = "tokenize,pos,lemma,depparse,constituency") 

2022-09-19 11:50:04 INFO: Downloading default packages for language: en (English)...
2022-09-19 11:50:06 INFO: File exists: C:\Users\antoi\stanza_resources\en\default.zip
2022-09-19 11:50:09 INFO: Finished downloading models and saved to C:\Users\antoi\stanza_resources.


2022-09-19 11:50:10 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| lemma        | combined |
| depparse     | combined |
| constituency | wsj      |

2022-09-19 11:50:10 INFO: Use device: cpu
2022-09-19 11:50:10 INFO: Loading: tokenize
2022-09-19 11:50:10 INFO: Loading: pos
2022-09-19 11:50:11 INFO: Loading: lemma
2022-09-19 11:50:11 INFO: Loading: depparse
2022-09-19 11:50:11 INFO: Loading: constituency
2022-09-19 11:50:11 INFO: Done loading processors!


# Functions

In [13]:
def make_POS_basics(tree, dico):
    """
    input :
    - tree = classic NLTK tree, with X-POS and terminal leaves that are the words of the sentence
    - dico = Stanza dictionnary https://stanfordnlp.github.io/stanza/data_objects.html#document
    
    returns :
    - simple_tree = simplified NLTK tree, with U-POS and NO words from the sentence
    """
    
    simple_tree = tree.copy(deep=True)
    token_numero = 0
    for position in tree.treepositions():
        try: 
            subtree = tree[position]
            if (type(subtree)==nltk.tree.Tree and subtree.height()==2) :
                newPOS = dico[token_numero]['upos']
                if ('"' in newPOS) or ('``' in newPOS) :
                  newPOS = 'QUOTESYMBOL'
                newsubtree = Tree.fromstring('(' + newPOS + ')')
                simple_tree[position] = newsubtree
                token_numero += 1
        except:
            continue
    return simple_tree

In [14]:
def C_tree_to_dot(t):
    """
    input :
    t =  NLTK Tree, 
    
    returns :
    s = a dot representation, suitable for using with Graphviz
    (type(s) = string for Python)
    """
    def gv_print(t,start_node=1):
        """
        Print the tree for a defined node. Nodes are specified in-order in the original tree	
        """
        # Print the start node of the tree
        s ='%s [label="%s"]' % (start_node,t.label())
        pos=start_node+1


        for child in t:
            if isinstance(child,nltk.tree.Tree): # the node is the root of a subtree ( non terminal )
                (s_child,newpos)=gv_print(child,pos)
                s=s+'\n'+ s_child
                s=s+'\n%s -> %s [dir=none]' % (start_node,pos)
                pos=newpos
            elif isinstance(child, str):  # the node is a leaf ( terminal )
                s=s+'\n%s [label="%s"]' % (pos,child)
                s=s+'\n%s -> %s' % (start_node,pos)	
            pos+=1
        return (s,pos-1)

    # Print the digraph dot specification
    s='digraph G{\n'	
    s+=gv_print(t)[0]
    s+="\n}"

    return s

In [15]:
def dot_to_nxgraph(path):
    """
    input :
    - path = directory + filename, filename is the name of the saved dot file
    
    returns :
    - final_G1 = a NeworkX Digraph, not rooted
    """
    G1 = nx.Graph(nx.nx_pydot.read_dot(path))
    label1 = nx.get_node_attributes(G1, 'label')
    final_G1 = nx.DiGraph()
    
    for vertex in G1.nodes.data('label'):
        final_G1.add_node(vertex[0], label=vertex[1])
    for edge in G1.edges.data():
        final_G1.add_edge(edge[0], edge[1])

    return final_G1

In [16]:
def plot_tree(T1, path):
  """
  input :
  - T1 = a NetorkX tree,
  - path = directory + filename

  returns :
  None, saves a png @ path
  """

  plt.figure(figsize=(8,8))
  pos = graphviz_layout(T1, prog="dot")
  f = nx.draw_networkx(T1, pos, arrows=False, with_labels=True, font_size=8) 
  f = nx.draw_networkx_labels(T1, pos, nx.get_node_attributes(T1, 'label'), verticalalignment='bottom', horizontalalignment='right', font_size=13, font_color='r',)
  plt.savefig(path+'.png')

In [29]:
def graph_n_tree(phrase, path, display=False):
  """
  input : 
  - phrase = string sentence
  - path =  directory + filename, filename is the name of the saved dot file
  - display = if True, displays the NLTK tree

  returns :
  - sent_dico = Stanza dictionnary https://stanfordnlp.github.io/stanza/data_objects.html#document
  - dot_tree = the dot file (type(dot_tree) = string for Python),
  - G = a NetworkX Directed Graph
  - T = a NetworkX Tree
  """
  doc = StanzaPipeline(phrase) #Stanza.Document
  sentence = doc.sentences[0]  #Stanza.Sentence
  sent_dico = sentence.to_dict()  #dictionnary
  stanza_tree = sentence.constituency #Stanza.Tree
  nltk_tree = Tree.fromstring(str(stanza_tree)) #NLTK.Tree

  if display :
    nltk_tree.pretty_print()

  dot_tree = C_tree_to_dot(nltk_tree)
  f = open(path+'.dot', 'w')
  f.write(dot_tree)
  f.close()
  G = dot_to_nxgraph(path+'.dot')
  T = nx.dfs_tree(G, source='1',)
  nx.set_node_attributes(T, dict(G.nodes(data=True)))

  return sent_dico, dot_tree, G, T

In [23]:
def write_ted_dot(span1, span2, dot1, dot2, m_score, c_score, edit_seq, path):
  """
  This function creates a .dot file to vizualise the mapping between the two trees.
  Relies on the .dot files of each textspan, which are basically concatenated.
  On top of that, a title and metrics is prepend, and the mapping is append
  (Deleted and Inserted nodes are marqued in red, Mapped ones in green).
  
  input :
  - span1/span2 = string textspan 1/2
  - dot1/dot2 = dot descriptions of textspan 1/2 (obtained from graph_n_tree())
  - m_score = first metric to display
  - c_score = second metric to display
  - edit_seq = edit path obtained during TED process @ format [(1,1), (2,4), (3, None) ...]
  - path =  directory + filename, filename is the name of the saved dot file
  
  returns :
  Nothing
  """  
  #zss_r = compute_zss(line.G1, line.G2)
  string = 'digraph G {\n'
  string += 'label="'+ span1.replace('"','') + '\n'
  string += span2.replace('"','') + '\n'
  string += 'RelSim = ' + str(round(m_score, 3)) + '\n'
  string += 'RelComp = ' + str(round(c_score, 3)) + '"\n'
  string += 'labelloc = t; fontname = "Helvetica,Arial,sans-serif"; fontsize = 20; \n'
  string += 'overlap = false; \n'
  string += 'splines = true; \n'
  string += 'subgraph cluster0 {\n'
  string += 'style=filled; color=lightgrey; label = "span 1"\n'
  contentG1 = dot1[11:-1]
  list_of_str = re.findall("[0-9]+", contentG1)
  list_of_int = [int(s) for s in list_of_str]
  max = np.max(list_of_int)
  string += contentG1 + '}\n'

  string += 'subgraph cluster1 { \n'
  string += 'style=filled; color=lightgrey; label = "span 2"\n'
  contentG2 = dot2[11:-1]
  def add(re_match):
    begin = re_match.span()[0]
    end = re_match.span()[-1]
    new = int(contentG2[begin:end]) + max
    return str(new)
  new_contentG2 = re.sub("[0-9]+", add, contentG2)
  string += new_contentG2 +'}\n'
  
  for u,v in edit_seq[1:] :
            if u == None and v != None :
                string += str(int(v)+max) + ' [color=firebrick3] \n'
            elif v == None : 
                string += str(u) + ' [color=firebrick3] \n'
            else :
                string += str(u) + ' -> ' + str(int(v)+max) + ' [color=chartreuse3, constraint=false] \n'


  string += '}'

  f = open(path+'.dot', 'w')
  f.write(string)
  f.close()


In [39]:
def get_path_edist(ALIGN):
  """
  input :
  - ALIGN = backtracing sequence at EDIST format https://edist.readthedocs.io/en/latest/ted.html
  
  returns :
  - the list of tuple of the match necessary for EDIST library @ format [(1,1), (2,4), (3, None) ...]
  """
  NUM_SEQ = []
  for TUPLE in ALIGN :
    left = TUPLE._left +1
    right = TUPLE._right +1
    if left == 0 :
      left = None
      right = str(right)
    elif right == 0 :
      right = None
      left = str(left)
    else :
      left = str(left)
      right = str(right)

    NUM_SEQ.append((left,right))
  
  return NUM_SEQ

In [20]:
def nxTree_to_edist(TREE):
  """
  input :
  - TREE = NetworkX tree
  
  returns :
  (two elements for creating a tree at EDIST format:
  https://gitlab.ub.uni-bielefeld.de/bpaassen/python-edit-distances/-/tree/master/ )
  - list_label = the node list
  - adj_list = the adjacency list
  """
  list_label = []
  for NODE in TREE.nodes :
    list_label.append(TREE.nodes[NODE]['label'][1:-1])


  adj_list = []
  for CHILDREN in nx.generate_adjlist(TREE):
    NEIGHBORS = re.findall("[0-9]+",CHILDREN)
    NEIGHBORS = [int(s)-1 for s in NEIGHBORS]
    adj_list.append(NEIGHBORS[1:])
  
  return list_label, adj_list


# sentences 

In [26]:
arg1 = "Nick makes the most delicious cakes"
arg2 = "and Grace makes the most disgusting desserts."

In [27]:
arg3= "Nick is excellent at baking"
arg4 = "and Grace makes the most disgusting desserts."

# Pipeline

In [1]:
def pipeline(spanA, spanB, PATH, DISPLAY=False):
    """
    this function compiles all above functions in order to compute the TED
    
    input :
    - spanA/spanB = textspan 1/2
    - PATH = concerns the mappring: directory + filename
    - display = to display the *full* syntax trees in the script
    
    returns:
    - RelSim and RelComp, or any metric of your choice
    """
    
    dictionnary1, dot1, G1, T1 = graph_n_tree(spanA, 'CFspan1', display=DISPLAY)
    dictionnary2, dot2, G2, T2 = graph_n_tree(spanB, 'CFspan2', display=DISPLAY)

    x_nodes, x_adj = nxTree_to_edist(T1)
    y_nodes, y_adj = nxTree_to_edist(T2)

    # define the distance between nodes in algebraic expressions
    def delta(x, y):
      if (x is None or y is None) :
        return 1.
      else :
        if(x == y):
          return 0.
        else:
          return 1e6 # or np.inf
    
    # ted.ted only computes the TED metric
    #TED = ted.ted(x_nodes, x_adj, y_nodes, y_adj, delta)
    
    # while ted.ted_backtrace returns the mapping for nodes of T1 to nodes of T2
    ALIGN = ted.ted_backtrace(x_nodes, x_adj, y_nodes, y_adj, delta)
    NUM_SEQ = get_path_edist(ALIGN)

    # To compute RelSim
    MatchedPairs = [pair for pair in NUM_SEQ[1:] if None not in pair]
    M = len(MatchedPairs)
    RelSim = M / min(G1.size(), G2.size())
    
    # To compute RelComp
    NodesSubset1 = [pair[0] for pair in MatchedPairs if (pair[0] != '1')]
    NodesSubset2 = [pair[1] for pair in MatchedPairs if (pair[0] != '1')]
    graph1 = G1.to_undirected()
    graph2 = G2.to_undirected()  
    forest1 = graph1.subgraph(NodesSubset1)
    forest2 = graph2.subgraph(NodesSubset2)
    ConnectedComponents1 = list(nx.connected_components(forest1))
    ConnectedComponents2 = list(nx.connected_components(forest2))
    ConnectedComponents1.sort(key = len)
    ConnectedComponents2.sort(key = len)
    RelComp = min(len(ConnectedComponents1[-1]),
            len(ConnectedComponents2[-1])) / M 
    
    # to get a representation of the mapping
    write_ted_dot(spanA, spanB, dot1, dot2, 
                  edit_seq=NUM_SEQ, 
                  c_score = RelComp, m_score = RelSim, 
                  path = PATH) 
    (graph,) = pydot.graph_from_dot_file(PATH+'.dot')
    graph.write_png(PATH+'.png')
    #Image(path + 'compare.png' )
    
    return RelSim, RelComp

In [43]:
RelSim, RelComp = pipeline(arg3, arg4, PATH = 'notcakes', DISPLAY=True)
print('\n=========================================')
print('Relative similarity = ', RelSim)
print('Relative compacity = ', RelComp)
print('=========================================')


            ROOT                      
             |                         
             S                        
  ___________|______                   
 |                  VP                
 |     _____________|___               
 |    |                ADJP           
 |    |       __________|____          
 |    |      |               PP       
 |    |      |           ____|____     
 NP   |      |          |         VP  
 |    |      |          |         |    
NNP  VBZ     JJ         IN        NN  
 |    |      |          |         |    
Nick  is excellent      at      baking

                ROOT                                  
                 |                                     
                 S                                    
  _______________|__________________________________   
 |    |          VP                                 | 
 |    |      ____|____                              |  
 |    |     |         NP                            | 
 |    |     |     _